<a href="https://colab.research.google.com/github/martinopiaggi/summarize/blob/main/Martino_Summarize_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarization notebook with AIs

---



Repository: https://github.com/martinopiaggi/summarize

In [20]:
Source = "Have you ever felt like you were in a zen-like meditative state while working, mentally free to execute and apply your skills? This is called being in the zone or getting inthe flow. No concern, no conscience of time\u2014just pure, invigorating work driven by heightened awareness.The term \u2018flow\u2019 was coined by Dr Mihaly Csikszentmihalysi in the 1960s. In his book Flow: The Psychology of Optimal Experience, he describes it as a highly focused mental state which facilitates productivity. He found that there was no correlation between money and happiness.The goal itself can be anything from learning a language, doing great work, sports, or playing an instrument. You could even get in the flow while doing the dishes. The only factors are that the goal needs to require your fullest attention and to provide clear cues to measure your progress.decide to focus on an activity for intrinsic reasons. Do the task for your own sake, and not because you feel obligated to. To achieve a flow state, you need to find the right balance between the challenge of the a and the pleasure of it. The steps below can be used to turn any task into a mini-meditation.ctivity and your own skills levels. The flow state rises from a tension between the two. You won\u2019t be able to get in the zone if the task is too easy or too difficult. Cut out distractions: this means no phone, no multitasking, only the necessary tools.Focus on the present moment, accept that you may fail, and let go of your ego. Work mindfully: keep your mind fully attentive to what you are working on in that moment. If you feel like it\u2019s wandering or slipping off into autopilot mode, gently bring it back.You don't have to be an athlete to benefit from the flow state. These five steps can help you stay relaxed and focused. Try to go through them next time you want to work on something or do an activity. \u201CFlow is important both because it makes the present instant more enjoyable, and because It builds the self-confidence that allows us to develop skills and make significant contributions to humankind\u201D" #@param {type:"string"}
Type_of_source = "Text" #@param ['Text', 'Youtube video or playlist', 'Videos on Google Drive folder','Dropbox video link']

Type = Type_of_source

if Type == 'Youtube video or playlist':
  URL = Source

if Type == 'Dropbox video link':
  dropbox_URL = Source

#@markdown ---
#@markdown ## Advanced settings

#@markdown If source is video, you want timestamps in final summary?
Timestamps = False #@param {type:"boolean"}

#@markdown Number of iterations of the summarization, recommended is 1
Iterations = 1 #@param {type:"number"}



In [3]:
!pip install transformers
!pip install tensorflow
from transformers import pipeline,BartTokenizer, BartForConditionalGeneration
from torch.utils.data import Dataset, DataLoader

import re
import math

if Type == ("Youtube video or playlist"
            or 'Videos on Google Drive folder'
            or "Dropbox video link"):

  !pip install faster-whisper
  from faster_whisper import WhisperModel
  from pathlib import Path
  import subprocess
  import torch
  import shutil
  import numpy as np

  if Type == "Youtube video or playlist":
    !pip install yt-dlp
    from pathlib import Path
    import yt_dlp

  if Type == ("Dropbox video link"):
    !sudo apt update && sudo apt install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
if Type == "Youtube video or playlist":
  video_path_local_list = []

  ydl_opts = {
  'format': 'm4a/bestaudio/best',
  'outtmpl': '%(id)s.%(ext)s',
  # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
  'postprocessors': [{  # Extract audio using ffmpeg
  'key': 'FFmpegExtractAudio',
  'preferredcodec': 'wav',
  }]
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download([URL])
    list_video_info = [ydl.extract_info(URL, download=False)]

  for video_info in list_video_info:
    video_path_local_list.append(Path(f"{video_info['id']}.wav"))

  #for video_path_local in video_path_local_list:
   # if video_path_local.suffix == ".mp4":
    #    video_path_local = video_path_local.with_suffix(".wav")
    #result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])


if Type == ("Dropbox video link"):
    !wget -O dropbox_video.mp4 $dropbox_URL
    !ffmpeg -i dropbox_video.mp4 -vn -acodec pcm_s16le -ar 16000 -ac 1 dropbox_video_audio.wav


[youtube] Extracting URL: https://www.youtube.com/watch?v=ojHwMPq8TYA
[youtube] ojHwMPq8TYA: Downloading webpage
[youtube] ojHwMPq8TYA: Downloading ios player API JSON
[youtube] ojHwMPq8TYA: Downloading android player API JSON
[youtube] ojHwMPq8TYA: Downloading player 0b285984
[youtube] ojHwMPq8TYA: Downloading m3u8 information
[info] ojHwMPq8TYA: Downloading 1 format(s): 140
[download] Destination: ojHwMPq8TYA.m4a
[download] 100% of   17.35MiB in 00:00:00 at 19.25MiB/s  
[FixupM4a] Correcting container of "ojHwMPq8TYA.m4a"
[ExtractAudio] Destination: ojHwMPq8TYA.wav
Deleting original file ojHwMPq8TYA.m4a (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=ojHwMPq8TYA
[youtube] ojHwMPq8TYA: Downloading webpage
[youtube] ojHwMPq8TYA: Downloading ios player API JSON
[youtube] ojHwMPq8TYA: Downloading android player API JSON
[youtube] ojHwMPq8TYA: Downloading m3u8 information


In [5]:
language = "en"
initial_prompt = ""
Text = ""

video_path_local = str(video_path_local_list[0])

if Type == ("Dropbox video link"):
    video_path_local = "dropbox_video_audio.wav"


if Type is not ("Text" or "Text from Google Drive"):

  model = WhisperModel('small', device="cuda", compute_type='int8')
  segments, info = model.transcribe(str(video_path_local), beam_size=5,
                                    language=None if language == "auto" else language,
                                    task="translate",
                                    initial_prompt=initial_prompt)

  transcript_file_name = video_path_local.replace(".wav", ".txt")
  with open(transcript_file_name, 'w') as f:
    for segment in segments:
      Text += segment.text.strip()
    f.write(Text+". ")


In [22]:
#Clean cache to avoid Colab crash in case of multiple runs
torch.cuda.empty_cache()
#Whisper model no longer necessary
del model
#load model
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to("cuda:0")

if Type is not ("Text" or "Text from Google Drive"):
  Text = open(transcript_file_name, "r").read()
  if Type == "Dropbox video link":
    Text = open("dropbox_video_audio.txt", "r").read()

if Type == "Text":
  transcript_file_name = "Text.txt"
  Text = Source

# Define the TextDataset
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]


for i in range(Iterations):

  # Tokenize the text and split into batches
  #texts = [Text[i:i+512] for i in range(0, len(Text), 512)]
  texts = [Text[i:i+512] for i in range(0, len(Text) - 100, 512 - 100)]

  dataset = TextDataset(texts)
  dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

  summary = ''

  for batch in dataloader:

      # Convert the batch to the appropriate format
      encoded_inputs = tokenizer(batch, truncation=True, padding=True, return_tensors="pt", max_length=512).to("cuda:0")

      summaries_output = model.generate(
          input_ids=encoded_inputs["input_ids"],
          attention_mask=encoded_inputs["attention_mask"],
          num_beams=4,
          length_penalty=0.6,
          no_repeat_ngram_size=2
      )

      # Decode each summarized output and append to the final summary
      for idx, output in enumerate(summaries_output):
          decoded_summary = tokenizer.decode(output, skip_special_tokens=True)
          summary += decoded_summary

  #in case of two or more iterations
  Text = summary

# Save the summary

final_name = 'summary_'
if Type != "Dropbox video link":
  final_name += transcript_file_name
else:
    final_name += "dropbox_video_audio.txt"

with open(final_name, 'w') as f:
    f.write(summary)